In [ ]:
import pandas as pd
import seaborn as sns
import csv

In [ ]:
file_path = 'user_courses_review_09_2023.csv'

data_list = []

EXPECTED_COLUMNS = 4  

    for row in reader:
        if len(row) == EXPECTED_COLUMNS:
            data_list.append(row)
        else:
            print(f"Skipping row {reader.line_num}: {row}")



In [ ]:
reader = csv.reader(file)

In [ ]:
header = next(reader)

In [ ]:
data = pd.DataFrame(data_list, columns = header)

print data.head()

In [ ]:
data_types = data.dtypes

if data['review_rating'].dtype =='object':
    data['review_rating'] = pd.to_numeric(data['review_rating'], errors='coerce')

new_data_types = data.dtypes
missing_values = data.isnull().sum()

data_types, new_data_types, missing_values 

In [ ]:
import matplotlib.pyplot as plt

data['review_ratings'].describe()

plt.hist（data['review_ratings']).dropna(),bins = range(1,7),align ='left', 

plt.title('Distribution of Review Ratings')
plt.xlabel('Rating'）
plt.ylabel('Frequency'）
plt.xticks(range(1,6))
plt.grid(axis='y')
plt.show()           



In [ ]:
clean_data = data.dropna()

new_shape = clean_data.shape

clean_data['comment_length']=clean_data['review_comment'].apply(len)
correlation = clean_data[['ratings'],['comment_length']].corr()

correlation



In [ ]:
import re

patterns = {
    "positive_emphasis": re.compile(r'\b(great|good|exellent|amazing|love)\b', re.IGNORECASE),
    "negative_emphasis": re.compile(r'\b(bad|terrible|awful|poor|worst|hate|disappoint)\b', re.IGNORECASE),
    "question": re.compile(r'\?'),
    "exclamation": re.compile(r'!'),
    "capital_words": re.compile(r'\b[A-Z]{2,}\b')  # words with all letters capitalized, and at least two characters long
}


pattern_counts = {key: 0 for key in patterns}

                  

for review in clean_data[['review_comment']:
    for key, pattern in patterns.items():
        if re.search(pattern, review):
            pattern_counts[key]+= 1
                         
                         
pattern_counts                         
    
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfiVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
import string


In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('','',string.punctuation)）
    return text
                          
lean_data.loc[:, 'review_comment'] = clean_data.loc[:,'review_comment']apply(clean_text)                         
                          
                          
X = clean_data.loc[:,'review_comment']
Y = clean_data.loc[:,'ratings']                         
                          
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,  random_state=42)
                          

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')

model = MultinomialNB()

text_clf = make_pipeline(vectorizer, model)

text_clf.fit(X_train, Y_train)

Y_pred =  text_clf.predict(X_test)

report = classification_report(Y_test, Y_pred) 

conf_matrix = confusion_matrix(Y_test, Y_pred) 

report, conf_matrix

In [ ]:
from sklearn.utils import resample

df_majority = clean_data[clean_data.review_rating == 5]
df_minority = clean_data[clean_data.review_rating < 5]

df_minority_upsampled = resample(df_minority,
                                 replace=True, 
                                 n_sample=len(df_majority),
                                 random_state=42)

df_upsampled =  pd.concat([df_majority, df_minority_upsampled'])
                           
df_upsampled.review_rating.value_count()                          


In [ ]:
X_upsampled = df_upsampled['review_comment']
Y_upsampled = df_upsampled['review_rating']

X_train_upsampled, X_test_upsampled, y_train_upsampled, y_test_upsampled = train_text_split(X_upsampled, Y_upsampled, test_size = 0.2, random_state = 42)

text_clf_upsampled = make_pipeline(vectorizer, model)
text_clf_upsampled.fit(X_train_upsampled, Y_train_upsampled） 

Y_predict_upsampled = text_clf_upsampled.predict(X_test_upsampled)                       

report_upsampled = classification_report(y_test_upsampled, y_pred_upsampled)

conf_matrix_upsampled = confusion_matrix(y_test_upsampled, y_pred_upsampled)                      

report_upsampled,  conf_matrix_upsampled                      

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(conf_matrix_upsampled, annot=True, fmt='d', cmap='Blues', xticklables=['1', '2', '3', '4', '5'], yticklables=['1', '2', '3', '4', '5'])
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title('Confusion Matrix')
plt.show()